In [ ]:
import os
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)


---
# <center><b>1. 파일 불러오기</b></center>
---

In [ ]:
# 파일 경로 지정하기
os.chdir('C:\민원분석참조모델\data\결과')

In [ ]:
# 결과 폴더 파일 목록 가져오기
df_list = list(os.listdir(os.getcwd()))
df_list

In [ ]:
# 몇번째 파일 분석할건지 지정
index = 0

In [ ]:
# 분석할 파일 불러오기
df = pd.read_csv(df_list[index], encoding='euc-kr')

In [ ]:
# 파일 개요
df.info()

In [ ]:
# 접수 경로 데이터 확인
df['접수경로']

----
# <center><b>2. 데이터 전처리</b></center>
---

In [ ]:
# 접수경로 빈도 추출
receptionPath = df.groupby(df['접수경로'], as_index=False).size()

In [ ]:
# 접수유형, 접수기관 컬럼 만들기
kind = list()  # 접수유형 리스트
agency = list()  # 접수기관 리스트

# 접수경로에서 특정 문자열 이상 드롭 (아웃라이어)
for j in receptionPath.index:
    if(len(receptionPath['접수경로'][j]) > 21):
        receptionPath = receptionPath.drop(j)

In [ ]:
# 접수유형, 접수기관 열 만들기
for k in receptionPath.index:
    num = receptionPath['접수경로'][k].find('(')  # ( 이전 문자열까지 자르기
    if(num == -1):
        kind.append(receptionPath['접수경로'][k])
        agency.append(np.nan)
    else:
        kind.append(receptionPath['접수경로'][k][:num])  # 자른 문자열 리스트에 추가
        agency.append(receptionPath['접수경로'][k][num+1:-1])

In [ ]:
# 열 붙이기
receptionPath['접수유형'] = kind
receptionPath['접수기관'] = agency

In [ ]:
# 열 이름 변경
receptionPath = receptionPath.rename(columns={'size': '빈도'})

In [ ]:
# 컬럼 순서 지정
receptionPath = receptionPath[['접수경로', '접수유형', '접수기관', '빈도']]

In [ ]:
# 접수경로 전처리 내용 확인
receptionPath.fillna('내용없음', inplace=True)
receptionPath

In [ ]:
# 파일 저장 - 접수경로 총 파일
receptionPath.to_csv(f'../../result/1.접수경로/접수경로_빈도_{df_list[index]}', encoding='euc-kr', index=False)

In [ ]:
# 파일 저장 - 접수유형 빈도
receptionKind = receptionPath.groupby('접수유형', as_index=False).sum()
receptionKind.to_csv(
    f'../../result/1.접수경로/접수유형_빈도_{df_list[index]}', encoding='euc-kr', index=False)
receptionKind


In [ ]:
# 파일 저장 - 접수기관
receptionAgency = receptionPath.groupby(
    ['접수유형', '접수기관'], as_index=False).sum()
receptionAgency.to_csv(
    f'../../result/1.접수경로/접수기관_빈도_{df_list[index]}', encoding='euc-kr', index=False)
receptionAgency


----
# <center><b>3. 데이터 시각화</b></center>
---

In [ ]:
# 접수유형 파이차트

# 레이아웃 설정
layout_kind = {
    'title': {
        'text': '<b>접수유형 빈도 파이차트 분석</b>',
        'font': {
            'size': 20
        },
        'x': 0.5,
    },
    'showlegend':True,
    'legend': {
        'title': {
            'text': '<b>접수유형</b>',
            'font': {
                'size': 15
            },
        },
        'borderwidth':25, 
        'bordercolor': '#fff',
        'bgcolor': '#F0F8FF',
        'orientation': 'h',
    },
    # 'autosize': True
    'width': 600,
    'height': 600,
}

kind = go.Figure()

kind.add_trace(
    go.Pie(
        labels=receptionKind['접수유형'],
        values=receptionKind['빈도'],
    )
)

kind.update_traces(
    go.Pie(
        textposition='inside',
        textinfo='label+value+percent',
        textfont = {
            'size': 14,
        }
    )
)

kind.update_layout(layout_kind)
kind.show()

In [ ]:
# 접수유형에 따른 접수기관 파이차트

layout_agency = {
    'title': {
        'text': '<b>접수유형에 따른 접수기관 파이차트 분석</b>',
        'font': {
            'size': 20
        },
        'x': 0.5,
    },
    'width': 600,
    'height': 600,
}

agency = px.sunburst(
    receptionAgency,
    path=['접수유형', '접수기관'],
    values='빈도'
)

agency.update_layout(layout_agency)
agency.show()


In [ ]:
# 접수기관 - 국민신문고

df_gookmin = receptionAgency[receptionAgency['접수유형'] == '국민신문고접수']
df_gookmin.sort_values('빈도', inplace=True, ascending=False)
df_gookmin = df_gookmin[['접수기관', '빈도']]
df_gookmin.reset_index(drop=True, inplace=True)
df_gookmin.set_index('접수기관',inplace=True)

layout_gookmin = {
    'title': {
        'text': '<b>[국민신문고민원]</b> 접수기관에 따른 접수민원수',
        'x': 0.5,
        'font': {
            'size': 20
        }
    },
    'showlegend': False,
    'xaxis': {
        'title': '접수기관',
        

    },
    'yaxis': {
        'title': '접수 민원 수',
        'showticklabels': True,
        'dtick': 5000
    }
}

gookmin = go.Figure()

gookmin.add_trace(
    go.Bar(
        x=df_gookmin.index,
        y=df_gookmin['빈도'], name='접수기관',
        text=df_gookmin['빈도'], textposition='auto',
    )
)

gookmin.update_layout(layout_gookmin)
gookmin.show()


In [ ]:
# 접수기관 - 국민신문고

df_mobile = receptionAgency[receptionAgency['접수유형'] == '모바일접수민원']
df_mobile.sort_values('빈도', inplace=True, ascending=False)
df_mobile = df_mobile[['접수기관', '빈도']]
df_mobile.reset_index(drop=True, inplace=True)
df_mobile.set_index('접수기관', inplace=True)

layout_mobile = {
    'title': {
        'text': '<b>[모바일접수민원]</b> 접수기관에 따른 접수민원수',
        'x': 0.5,
        'font': {
            'size': 20
        }
    },
    'showlegend': False,
    'xaxis': {
        'title': '접수기관',


    },
    'yaxis': {
        'title': '접수 민원 수',
        'showticklabels': True,
        'dtick': 5000
    }
}

mobile = go.Figure()

mobile.add_trace(
    go.Bar(
        x=df_mobile.index,
        y=df_mobile['빈도'], name='접수기관',
        text=df_mobile['빈도'], textposition='auto',
    )
)

mobile.update_layout(layout_mobile)
mobile.show()


In [ ]:
# 접수기관 - 국민신문고

df_ja = receptionAgency[receptionAgency['접수유형'] == '자체접수민원']
df_ja.sort_values('빈도', inplace=True, ascending=False)
df_ja = df_ja[['접수기관', '빈도']]
df_ja.reset_index(drop=True, inplace=True)
df_ja.set_index('접수기관', inplace=True)

layout_ja = {
    'title': {
        'text': '<b>[자체접수민원]</b> 접수기관에 따른 접수민원수',
        'x': 0.5,
        'font': {
            'size': 20
        }
    },
    'showlegend': True,
    'xaxis': {
        'title': '접수기관',
    },
    'yaxis': {
        'title': '접수 민원 수',
        'showticklabels': True,
        # 'dtick': 5000
    }
}

ja = go.Figure()

ja.add_trace(
    go.Bar(
        x=df_ja.index,
        y=df_ja['빈도'], name='접수기관',
        text=df_ja['빈도'], textposition='auto',
    )
)

ja.update_layout(layout_ja)
ja.show()


In [ ]:
# 접수기관 - 국민신문고

df_related = receptionAgency[receptionAgency['접수유형'] == '연계기관접수']
df_related.sort_values('빈도', inplace=True, ascending=False)
df_related = df_related[['접수기관', '빈도']]
df_related.reset_index(drop=True, inplace=True)
df_related.set_index('접수기관', inplace=True)

layout_related = {
    'title': {
        'text': '<b>[연계기관접수]</b> 접수기관에 따른 접수민원수',
        'x': 0.5,
        'font': {
            'size': 20
        }
    },
    'showlegend': False,
    'xaxis': {
        'title': '접수기관',
        'tickangle': -45,
        'tickfont': {
            'size': 10
        },
    },
    'yaxis': {
        'title': '접수 민원 수',
        'showticklabels': True,
    },
    # 'width': 1000,
    'height': 500,
}

related = go.Figure()

related.add_trace(
    go.Bar(
        x=df_related.index,
        y=df_related['빈도'], name='접수기관',
        text=df_related['빈도'], textposition='auto',
        orientation='v',
    )
)

related.update_layout(layout_related)
related.show()


In [ ]:
# 접수기관 - 국민신문고

df_bok = receptionAgency[receptionAgency['접수유형'] == '복합민원']
df_bok.sort_values('빈도', inplace=True, ascending=False)
df_bok = df_bok[['접수기관', '빈도']]
df_bok.reset_index(drop=True, inplace=True)
df_bok.set_index('접수기관', inplace=True)

layout_bok = {
    'title': {
        'text': '<b>[복합민원]</b> 접수기관에 따른 접수민원수',
        'x': 0.5,
        'font': {
            'size': 20
        }
    },
    'showlegend': False,
    'xaxis': {
        'title': '접수기관',
        # 'tickangle': -45,
        # 'tickfont': {
        #     'size': 10
        # },
    },
    'yaxis': {
        'title': '접수 민원 수',
        'showticklabels': True,
    },
    # 'width': 1000,
    # 'height': 500,
}

bok = go.Figure()

bok.add_trace(
    go.Bar(
        x=df_bok.index,
        y=df_bok['빈도'], name='접수기관',
        text=df_bok['빈도'], textposition='auto',
        orientation='v',
    )
)

bok.update_layout(layout_bok)
bok.show()
